In [3]:
import os
import time
import numpy as np
import sys
import torch
import torch.optim as optim

os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
sys.path.append('../../../')
from agora.DAG.algorithm.gnnDeepRL.agent import Agent
from cogito.machine import MachineConfig
from agora.DAG.utils.csv_reader import CSVReader
from agora.auxiliary.tools import average_completion, average_slowdown
from agora.DAG.adapter.episode import Episode
from agora.DAG.algorithm.gnnDeepRL.DRL import RLAlgorithm
from agora.DAG.algorithm.gnnDeepRL.reward_giver import EnergyOptimisationRewardGiverV2
from agora.DAG.utils.feature_functions import *
from agora.DAG.algorithm.gnnDeepRL.utils import ExperienceBuffer
# Import the CloudResourceGNN and train_gnn_ac functions
from agora.DAG.algorithm.gnnDeepRL.brain import CloudResourceGNN, train_gnn_ac

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

np.random.seed(41)

# ************************ Parameters Setting Start ************************
machines_number = 5
jobs_len = 10
n_iter = 1
jobs_csv = '../jobs_files/batch_task.csv'

reward_giver = EnergyOptimisationRewardGiverV2()

name = f'GNN-{machines_number}'
model_dir = f'./agents/{name}'
summary_path = f'./Tensorboard/{name}'

# ************************ Parameters Setting End ************************

if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

csv_reader = CSVReader(jobs_csv)
machine_configs = [
                   MachineConfig(240, 1, 1, "ThinkSystem-SR850-V3",1900), MachineConfig(8, 1, 1, "HPProLiant-ML30-Gen11",3200),
                   MachineConfig(80, 1, 1, "FALINUX-AnyStor-700EC-NM",3000),MachineConfig(64, 1, 1, "Dell-PowerEdgeHS5610",2100), MachineConfig(120, 1, 1, "FusionServer-1288H-V7",1900)]
from agora.DAG.algorithm.gnnDeepRL.utils import *




length of dataframe:  98434


C:\Users\AH274303\Documents\GitHub\LETSDOIT\agora\DAG\launch_scripts\../../..\agora\DAG\utils\csv_reader.py:18: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.instances_num = df.inst_num.astype(dtype=int)


In [7]:
for job_chunk in range(4,5):
    jobs_configs = csv_reader.generate(jobs_len * job_chunk, jobs_len)

    for j in jobs_configs:
        print(j.submit_time)
        print(j.tasks_map)
        for k,task_config in j.tasks_map.items():
            task_config_cpu=task_config.cpu
            task_config_memory=task_config.memory
            task_config_duration=task_config.duration
            
        

Length of job_configs is  5
Jobs number:  5
Tasks number: 10
Task instances number mean:  54.7
Task instances number std 154.52446408255233
Task instances cpu mean:  52.376599634369285
Task instances cpu std:  21.88178395095497
Task instances memory mean:  0.5850822669104205
Task instances memory std:  0.06403368768881976
Task instances duration mean:  25.424131627056674
Task instances duration std:  8.37005211013451
0
{'2': <cogito.config.TaskConfig object at 0x0000017B66396150>, '1': <cogito.config.TaskConfig object at 0x0000017B66396210>}
11
{'a': <cogito.config.TaskConfig object at 0x0000017B657F4E10>}
13
{'a': <cogito.config.TaskConfig object at 0x0000017B668D1210>}
26
{'2': <cogito.config.TaskConfig object at 0x0000017B6514CB90>, '1': <cogito.config.TaskConfig object at 0x0000017B6514CC50>}
39
{'1': <cogito.config.TaskConfig object at 0x0000017B64DC35D0>, '4': <cogito.config.TaskConfig object at 0x0000017B64DC3690>, '3': <cogito.config.TaskConfig object at 0x0000017B64DC3750>, '2

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

for job_chunk in range(4,5):
    jobs_configs = csv_reader.generate(jobs_len * job_chunk, jobs_len)

    for job_index, j in enumerate(jobs_configs):
        # Create a list to store task data
        task_data = []

        for task_id, task_config in j.tasks_map.items():
            task_data.append({
                'Task ID': task_id,
                'CPU': task_config.cpu,
                'Memory': task_config.memory,
                'Duration': task_config.duration,
                'inst': task_config.inst_num
            })

        # Convert to DataFrame
        df = pd.DataFrame(task_data)

        # Create a figure with subplots
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 15))
        fig.suptitle(f'Job {job_index + 1} Task Analysis (Submit Time: {j.submit_time})')

        # 1. Bar plot for CPU requirements
        sns.barplot(x='Task ID', y='CPU', data=df, ax=ax1)
        ax1.set_title('CPU Requirements by Task')
        ax1.set_xlabel('Task ID')
        ax1.set_ylabel('CPU')

        # 2. Bar plot for Memory requirements
        sns.barplot(x='Task ID', y='Memory', data=df, ax=ax2)
        ax2.set_title('Memory Requirements by Task')
        ax2.set_xlabel('Task ID')
        ax2.set_ylabel('Memory')

        # 3. Scatter plot for CPU vs Memory, with size representing Duration
        sns.scatterplot(x='CPU', y='Memory', size='Duration', data=df, ax=ax3)
        ax3.set_title('CPU vs Memory (Size: Duration)')
        ax3.set_xlabel('CPU')
        ax3.set_ylabel('Memory')

        plt.tight_layout()
        
        # Save the plot
        plt.savefig(f'job_{job_index + 1}_analysis.png')
        plt.close()

    print(f"Plots for job chunk {job_chunk} have been saved.")

Length of job_configs is  5
Jobs number:  5
Tasks number: 10
Task instances number mean:  54.7
Task instances number std 154.52446408255233
Task instances cpu mean:  52.376599634369285
Task instances cpu std:  21.88178395095497
Task instances memory mean:  0.5850822669104205
Task instances memory std:  0.06403368768881976
Task instances duration mean:  25.424131627056674
Task instances duration std:  8.37005211013451
Plots for job chunk 4 have been saved.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

for job_chunk in range(0,15):
    jobs_configs = csv_reader.generate(jobs_len * job_chunk, jobs_len)

    for job_index, j in enumerate(jobs_configs):
        task_data = []

        for task_id, task_config in j.tasks_map.items():
            task_data.append({
                'Task ID': task_id,
                'CPU': task_config.cpu,
                'Memory': task_config.memory,
                'Duration': task_config.duration,
                'inst': task_config.instances_number 
            })

        df = pd.DataFrame(task_data)

        # Create a figure with subplots
        fig = plt.figure(figsize=(20, 25))
        fig.suptitle(f'Job {job_index + 1} Task Analysis (Submit Time: {j.submit_time})', fontsize=16)

        # 1. Bar plot for CPU requirements
        ax1 = fig.add_subplot(421)
        sns.barplot(x='Task ID', y='CPU', data=df, ax=ax1)
        ax1.set_title('CPU Requirements by Task')
        ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')

        # 2. Bar plot for Memory requirements
        ax2 = fig.add_subplot(422)
        sns.barplot(x='Task ID', y='Memory', data=df, ax=ax2)
        ax2.set_title('Memory Requirements by Task')
        ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
        
        # 2. Bar plot for Memory requirements
        ax2 = fig.add_subplot(423)
        sns.barplot(x='Task ID', y='inst', data=df, ax=ax2)
        ax2.set_title('number of instances Requirements by Task')
        ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
        
        # 4. Histogram of CPU requirements
        ax4 = fig.add_subplot(424)
        sns.histplot(data=df, x='CPU', kde=True, ax=ax4)
        ax4.set_title('Distribution of CPU Requirements')

        # 5. Histogram of Memory requirements
        ax5 = fig.add_subplot(425)
        sns.histplot(data=df, x='Memory', kde=True, ax=ax5)
        ax5.set_title('Distribution of Memory Requirements')

        # 6. Box plot of CPU and Memory
        ax6 = fig.add_subplot(426)
        df_melted = pd.melt(df, value_vars=['CPU', 'Memory'])
        sns.boxplot(x='variable', y='value', data=df_melted, ax=ax6)
        ax6.set_title('Box Plot: CPU and Memory')

        # 7. Heatmap of correlation between CPU, Memory, and Duration
        ax7 = fig.add_subplot(427)
        corr = df[['CPU', 'Memory', 'Duration']].corr()
        sns.heatmap(corr, annot=True, cmap='coolwarm', ax=ax7)
        ax7.set_title('Correlation Heatmap')

        # 8. Parallel coordinates plot
        ax8 = fig.add_subplot(428)
        pd.plotting.parallel_coordinates(df, 'Task ID', ax=ax8)
        ax8.set_title('Parallel Coordinates Plot')
        ax8.legend_.remove()

        plt.tight_layout()
        plt.subplots_adjust(top=0.95)
        
        # Save the plot
        plt.savefig(f'job_{job_chunk}{job_index + 1}_detailed_analysis.png', dpi=300, bbox_inches='tight')
        plt.close()

    print(f"Detailed plots for job chunk {job_chunk} have been saved.")

Length of job_configs is  10
Jobs number:  10
Tasks number: 32
Task instances number mean:  4069.375
Task instances number std 14894.458688028075
Task instances cpu mean:  0.958070956842267
Task instances cpu std:  0.20042704033540495
Task instances memory mean:  0.2962985716479804
Task instances memory std:  0.09579533581604799
Task instances duration mean:  88.5822454308094
Task instances duration std:  48.374854109833954


C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:31: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:37: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:43: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\anaconda3\envs\CloudEnvironement\Lib\site-packages\seaborn\matrix.py:202: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
C:\Users\AH2743

C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:37: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:43: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')


Detailed plots for job chunk 0 have been saved.
Length of job_configs is  10
Jobs number:  10
Tasks number: 55
Task instances number mean:  301.8181818181818
Task instances number std 1093.85239308201
Task instances cpu mean:  0.472289156626506
Task instances cpu std:  0.4992315185958613
Task instances memory mean:  0.44121686746987954
Task instances memory std:  0.09051573093564322
Task instances duration mean:  10.36987951807229
Task instances duration std:  25.91783697532377


C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:31: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:37: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:43: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:31: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLoc

...

Detailed plots for job chunk 7 have been saved.
Length of job_configs is  10
Jobs number:  10
Tasks number: 39
Task instances number mean:  362.05128205128204
Task instances number std 1292.7711826551072
Task instances cpu mean:  0.05524079320113314
Task instances cpu std:  0.22844966178053927
Task instances memory mean:  0.2128186968838527
Task instances memory std:  0.053353420937936505
Task instances duration mean:  21.53116147308782
Task instances duration std:  80.47373733130213


C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:31: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:37: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:43: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:31: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLoc

C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:43: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:31: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:37: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:43: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLoc

Detailed plots for job chunk 8 have been saved.
Length of job_configs is  10
Jobs number:  10
Tasks number: 69
Task instances number mean:  1498.2608695652175
Task instances number std 3801.1618957937094
Task instances cpu mean:  0.11375507835171213
Task instances cpu std:  0.3175135595544039
Task instances memory mean:  0.499063648674792
Task instances memory std:  0.0997955620253656
Task instances duration mean:  734.3797639775585
Task instances duration std:  522.7076383627922


C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:31: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:37: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\Temp\ipykernel_13900\1202726238.py:43: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45, ha='right')
C:\Users\AH274303\AppData\Local\anaconda3\envs\CloudEnvironement\Lib\site-packages\seaborn\matrix.py:202: RuntimeWarning: All-NaN slice encountered
  vmin = np.nanmin(calc_data)
C:\Users\AH2743